In [9]:

# !pip install nkocr --user
# !git clone https://github.com/a1xg/OCR-pipeline-for-product-labels.git
# !cd OCR-pipeline-for-product-labels
from transformers import AutoTokenizer, AutoModel, XLMRobertaTokenizer, XLMRobertaForQuestionAnswering
import torch
import cv2
from ocr import ImageOCR
import pytesseract
from nkocr import OcrProduct
import os
import warnings
import pandas as pd
warnings.filterwarnings("ignore")

pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract'

tokenizer = XLMRobertaTokenizer.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")
model = XLMRobertaForQuestionAnswering.from_pretrained("AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru")

rubert_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
rubert_model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
import re

def prepare_text(text):
    d = {'&': "8",
         '$': "8",
         '©': 'c',
         ';': ':',
         '«': '"',
         '»': '"'
         }
    text = text.replace('\\n', ' ')
    text = re.sub(r'\s', ' ', text)
    text = re.sub(r'\.{2,}', ' ', text)
    text = ''.join([d[s] if s in d else s for s in text])
    text = re.sub(r'[^ \.%"\(\):/,1234567890АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя]', '', text)
    return text


# получение текстов из картинки
def get_texts(file_path, printing=False, using_eng=False):
    image = cv2.imread(file_path)
    ocr = ImageOCR(image)
    try:
        recognized_text1 = ' '.join([x['text'] for x in ocr.get_text(text_lang='rus', crop=True)])
    except:
        recognized_text1 = ''
    recognized_text2 = str(OcrProduct(file_path, language='rus'))
    recognized_text3 = ''
    if using_eng:
        recognized_text3 = str(OcrProduct(file_path, language='eng'))
    recognized_text1, recognized_text2, recognized_text3 = prepare_text(recognized_text1), prepare_text(recognized_text2), prepare_text(recognized_text3)
    if printing:
        print(recognized_text1)
        print(recognized_text2)
        print(recognized_text3)
    return recognized_text1, recognized_text2, recognized_text3

In [11]:
# получение по текстам ответы на вопросы
# questions - list с вопросами
# context - list с текстами по каждой картинке
def get_answer(questions, context):
    new_context = []
    for el in context:
        for _ in range(len(questions)):
            new_context.append(el)
    for idx in range(len(new_context), len(questions)):
        new_context[idx] = new_context[idx][:50]
    
    new_questions = []
    for _ in range(len(context)):
        for el in questions:
            new_questions.append(el)

    inputs = tokenizer(new_questions, new_context, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_scores = outputs.start_logits.reshape(len(context), len(questions), -1)
    answer_start = torch.argmax(answer_start_scores, axis=2)

    sh = answer_start_scores.shape[-1]
    answer_end_scores = outputs.end_logits.reshape(len(context), len(questions), -1)*torch.clamp(((torch.ones_like(answer_start_scores) * torch.tensor(range(1, sh+1))).T.reshape(sh, -1) - answer_start.reshape(-1)).reshape(sh, len(context), len(questions)).permute(1, 2, 0), min=1)**0.25
    #answer_end_scores = outputs.end_logits.reshape(len(context), len(questions), -1)
    #if(answer_end_scores.shape[1] >= 4):
       # answer_end_scores[:, 4, :] = ((answer_end_scores[:, 4, :]**3 + ((torch.tensor(range(sh)).T)**0.666)).T -answer_start[:,4]**0.666).T
    answer_end = torch.argmax(answer_end_scores, axis=2) + 1

    confs = answer_start_scores.max(axis=2).values+outputs.end_logits.reshape(len(context), len(questions), -1).max(axis=2).values

    # Decode the answer tokens
    answers = [[] for _ in range(len(context))]
    inputs = inputs["input_ids"]
    for i in range(len(context)):
        for j in range(len(questions)):
            inp = inputs[len(questions)*i + j]
            start = answer_start[i][j].item()
            end = answer_end[i][j].item()
            answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inp[start:end]))
            answers[i].append(answer)

    return answers, confs


In [12]:

def get_questions_for_eng(settings):
    questions = []
    idxs = []
    for i, s in enumerate(settings):
        if s[1]:
            idxs.append(i)
            questions.append(s[0])
    return questions, idxs


def get_ans_conf_for_eng(ans3, conf3, idxs, context_length, questions_length):
    ans3_ = [['' for _ in range(questions_length)] for __ in range(context_length)]
    conf3_ = -torch.ones(context_length, questions_length) * float('inf')
    for j in range(context_length):
        for i, idx in enumerate(idxs):
            ans3_[j][idx] = ans3[j][i]
            conf3_[j][idx] = conf3[j][i]
    return ans3_, conf3_


def get_final_answers(dir_path, settings, printing=False): 
    file_pathes = [os.path.join(dir_path, x) for x in os.listdir(dir_path)]
    recognized_text1, recognized_text2, recognized_text3 = [], [], []
    for file_path in file_pathes:
        t1, t2, t3 = get_texts(file_path, printing=True, using_eng=True)
        recognized_text1.append(t1)
        recognized_text2.append(t2)
        recognized_text3.append(t3)
    questions = [s[0] for s in settings]
    questions_eng, idxs_eng = get_questions_for_eng(settings)
    ans1, conf1 = get_answer(questions, recognized_text1)
    ans2, conf2 = get_answer(questions, recognized_text2)
    ans3, conf3 = get_answer(questions_eng, recognized_text3)
    
    context_length, questions_length = conf2.shape
    ans3, conf3 = get_ans_conf_for_eng(ans3, conf3, idxs_eng, context_length, questions_length)
    #return (ans1, conf1), (ans2, conf2), (ans3, conf3)
    texts = [recognized_text1, recognized_text2, recognized_text3]
    answers = [ans1, ans2, ans3]
    confs = [conf1.unsqueeze(-1), conf2.unsqueeze(-1), conf3.unsqueeze(-1)]
    correct_nums_text = torch.concat(confs, axis=-1).argmax(axis=-1)
    correct_nums_image = torch.concat(confs, axis=-1).max(axis=-1).values.argmax(axis=0)
    best_confs = torch.concat(confs, axis=-1).max(axis=-1).values.max(axis=0).values
    correct_answers = []
    
    for i, num_image in enumerate(correct_nums_image):
        num_text = correct_nums_text[num_image][i]
        correct_answers.append(answers[num_text][num_image][i])
    if printing:
        print(correct_nums_text)
        print(correct_nums_image)
    idxs = []
    for i, s in enumerate(settings):
        if s[2]:
            idxs.append(i)
    idx = idxs[0]
    strong_text = texts[correct_nums_text[correct_nums_image[idx]][idx]][correct_nums_image[idx]]

    return correct_answers, best_confs, strong_text

In [13]:
from sklearn.metrics.pairwise import cosine_distances
def rubert(sentence):
    inputs = rubert_tokenizer(sentence, max_length=512, truncation=True, return_tensors='pt')
    with torch.no_grad():
        out = rubert_model(**inputs).last_hidden_state[0].mean(dim=0)
    return out.reshape(1, -1)

def compute_similarity(first, second):
    first, second = rubert(first), rubert(second)
    return 1-cosine_distances(first, second)


def get_category(strong_text, cats, printing=True):
    best_sim = -1
    for cat in cats:
        sim = compute_similarity(cat, strong_text).item()
        if sim > best_sim:
            best_sim = sim
            best_cat = cat
    best_cat = best_cat.split('.')[0]
    if printing:
        print('КАТЕГОРИЯ:')
        print(best_sim, best_cat)
    return best_sim, best_cat

In [14]:

# настройки предикта в формате:
# question - задаваемый вопрос, 
# use_eng - использовать ли модель для предикта на англ, 
# save_conf - учитывать ли при отборе лучшего текста для берта
settings = [
    ['Наименование продукта', True, False],
    ['С какого возраста можно употреблять этот продукт?', False, False],
    ['Какие вредные вещества содержатся в продукте?', False, False],
    ['Как называется компания изготовитель этого продукта?', True, False],
    ['Полный состав продукта', False, True],
    ['Какая информация об этом продукте является рекламной?', False, False],
    ['Какова энергетическая ценность продукта детского питания?', False, False],
    ['Какова пищевая ценность продукта?', False, False],
    ['Какие подсластители и свободные сахара содержатся в продукте?', False, False],
    ['Сколько натрия содержится в продукте?', False, False],
    ['Сколько сахара содержится в продукте?', False, False]
]


# тексты для определения категории по средству cosine similarity
cat1 = "Cухие каши и крахмалистые продукты. Сухая или порошкообразная каша (крахмалистый продукт), готовые к употреблению в пищу или требующие варки с молоком или водой. Требуется приготовление с добавлением молока (либо аналогичной неподслащённой жидкости) или воды (либо безбелковой жидкости). Например, каши быстрого приготовления, мюсли, рис для детского питания, сухие макаронные изделия."
cat2 = "Молочные продукты. Блюда на основе молочных продуктов, десерты и каши. Основной ингредиент – молочный продукт и фрукты (≤5%). Например, овсяная каша, рисовый пудинг, йогурт, мягкий сыр, заварной крем."
cat3_1 = "Фруктовые и овощные пюре/коктейли и фруктовые десерты. Продукт для завтрака, содержащий фрукты, и молочные продукты. Любой продукт, содержащий >5% фруктов1 (за исключением сухих каш, молочных продуктов с низким содержанием фруктов и перекусов). Например, яблочное пюре, йогурт с добавлением фруктов, заварной крем с фруктами, овсяная каша, содержащая >5% фруктов"
cat3_2 = "Фруктовые и овощные пюре/коктейли и фруктовые десерты. Продукт, содержащий только овощи. Монокомпонентные или поликомпонентные овощные, бобовые продукты. Например, пюре из шпината и гороха, картофельное пюре с морковью."
cat4_1 = "Поликомпонентные продукты/блюда: сочетания крахмалистых (крупяных) продуктов, овощей, молочных продуктов и/или белка. К источникам белка относятся любые виды мяса, мясных субпродуктов, птицы или рыбы. Продукты, в названии которых НЕ упомянуто содержание белка. Овощи/бобовые и/или каши/крахмалистые продукты. Может содержать источник белка, молочные продукты или жиры. Например, рис с овощами, лазанья, соус «песто» для макаронных изделий."
cat4_2 = "Поликомпонентные продукты/блюда: сочетания крахмалистых (крупяных) продуктов, овощей, молочных продуктов и/или белка. К источникам белка относятся любые виды мяса, мясных субпродуктов, птицы или рыбы. Продукты, в названии которых упомянуто содержание сыра. В названии продукта указан сыр, но не указаны другие источники белка. Например, макароны с сыром, соус для макаронных изделий с помидорами и сыром «моцарелла»."
cat4_3 = "Поликомпонентные продукты/блюда: сочетания крахмалистых (крупяных) продуктов, овощей, молочных продуктов и/или белка. К источникам белка относятся любые виды мяса, мясных субпродуктов, птицы или рыбы. Продукты, в которых источник белка указан, но НЕ упомянут в названии продукта первым. Источник белка не является первым указанным в названии продуктом. Например, карри с горошком и бараниной, соус для макаронных изделий с томатами и говядиной"
cat4_4 = "Поликомпонентные продукты/блюда: сочетания крахмалистых (крупяных) продуктов, овощей, молочных продуктов и/или белка. К источникам белка относятся любые виды мяса, мясных субпродуктов, птицы или рыбы. Продукты, в которых источник белка упомянут в названии продукта первым. Источник белка является первым указанным в названии продуктом. Например, кролик с картофелем, суп из говядины, вкусное ризотто с курицей, макаронные изделия с курицей и сыром, соус для макаронных изделий с говядиной"
cat4_5 = "Поликомпонентные продукты/блюда: сочетания крахмалистых (крупяных) продуктов, овощей, молочных продуктов и/или белка. К источникам белка относятся любые виды мяса, мясных субпродуктов, птицы или рыбы. Продукты, в которых упомянут только источник белкаГомогенизированное отварное мясо. Может содержать небольшое количество зерновых продуктов / крахмала, не указанных в названии продукта. Например, «Кролик» или «Ягнёнок» с добавлением небольшого количества рисовой или кукурузной муки."
cat5_1 = "Сухие закуски и перекусы. Фрукты. Включает свежие цельные или очищенные фрукты и сушеные фрукты. Например, сушёные яблочные ломтики или изюм."
cat5_2 = "Сухие закуски и перекусы. Сухие закуски или перекусы, которые едят руками. Любые перекусы, состоящие из зерновых, крахмалистых продуктов, бобовых/чечевицы или корнеплодов, такие как крекеры, хлеб, печенье, выпечка, пирожные или блинчики и т.д. К этой категории относятся также сухарики, крекеры и печенье, которые следует употреблять в сухом или размоченном виде"
cat6 = "Ингредиенты. Ингредиенты для приготовления пищи и добавления в продукты в небольших количествах. Например, оливковое масло, бульонные кубики."
cat7 = "Кондитерские изделия. К кондитерским изделиям относятся: шоколад, конфеты, лакрица, марципан, жевательная фруктовая пастила и т.д."
cat8 = "Напитки. Фруктовый сок и другие подслащённые или ароматизированные напитки."
cats = [cat1, cat2, cat3_1, cat3_1, cat4_1, cat4_2, cat4_3, cat4_4, cat4_5, cat5_1, cat5_2, cat6, cat7, cat8]


In [15]:
dir_path = 'agusha'
printing = True
correct_answers, best_confs, strong_text = get_final_answers(dir_path, settings, printing=printing)
best_sim, best_cat = get_category(strong_text, cats, printing=printing)

final_dict = {
        'Путь к папке с картинками': [dir_path],
        'Результат работы OCR': [strong_text],
        'Категория продукта': [best_cat]
    }
for i in range(len(settings)):
    q = settings[i][0]
    a = correct_answers[i] if best_confs[i]>=3 else None
    final_dict[q] = [a]

df = pd.DataFrame(final_dict)
print('confs', best_confs)
if best_confs[2]>=4:
    df['Содержатся ли вредные вещества в продукте?'] = ['ДА']
else:
    df['Содержатся ли вредные вещества в продукте?'] = ["НЕТ"]

if best_confs[8]>=4:
    df['Содержатся ли подсластители или свободные сахара в продукте?'] = ['ДА']
else:
    df['Содержатся ли подсластители или свободные сахара в продукте?'] = ["НЕТ"]

if df['Содержатся ли вредные вещества в продукте?'].values[0] == "ДА" or df['Содержатся ли подсластители или свободные сахара в продукте?'].values[0] == "ДА":
    df['Пригоден ли продукт для детей?'] = ["НЕТ"]
else:
    df['Пригоден ли продукт для детей?'] = ["ДА"]
df.to_csv(f'{dir_path}_product_info.csv', index=False)
df

-l rus --oem 1 --psm 6
tensor([[2, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
КАТЕГОРИЯ:
0.5640671849250793 Cухие каши и крахмалистые продукты
confs tensor([ 4.1926, 13.2926,  5.5208,  5.7442,  5.8545,  5.8459,  6.5469,  8.1197,
         4.5034,  5.3598,  6.0098])


,Путь к папке с картинками,Результат работы OCR,Категория продукта,Наименование продукта,С какого возраста можно употреблять этот продукт?,Какие вредные вещества содержатся в продукте?,Как называется компания изготовитель этого продукта?,Полный состав продукта,Какая информация об этом продукте является рекламной?,Какова энергетическая ценность продукта детского питания?,Какова пищевая ценность продукта?,Какие подсластители и свободные сахара содержатся в продукте?,Сколько натрия содержится в продукте?,Сколько сахара содержится в продукте?,Содержатся ли вредные вещества в продукте?,Содержатся ли подсластители или свободные сахара в продукте?,Пригоден ли продукт для детей?
0,agusha,"ЙОГУРТ ""АГУША"" С КЛУБНИКОЙ ИБАНАНОМ, ОБОГАЩЕНН...",Cухие каши и крахмалистые продукты,"ЙОГУРТ ""АГУША""",СТАРШЕ 8 МЕСЯЦЕВ,КАЛОРИЙНОСТЬ) акция проводится на территории Р...,"ЙОГУРТ ""АГУША""",молоко нормализованное,КАЛОРИЙНОСТЬ,СРЕДНИЕ ЗНАЧЕНИЯ): (КАЛОРИЙНОСТЬ) акция провод...,ВВ12 ПИЩЕВАЯ ЦЕННОСТЬ В 100 Г ПРОДУКТА (СРЕДНИ...,"ароматизаторы натуральные (""Клубника"", ""Банан""...",1108 КОЕ/,1108 КОЕ/,ДА,ДА,НЕТ


In [16]:
choised_cats = ['Категория продукта', 'Наименование продукта', 'С какого возраста можно употреблять этот продукт?', 'Какие вредные вещества содержатся в продукте?', 'Как называется компания изготовитель этого продукта?', 'Полный состав продукта', 'Какая информация об этом продукте является рекламной?', 'Какова энергетическая ценность продукта детского питания?', 'Какова пищевая ценность продукта?', "Какие подсластители и свободные сахара содержатся в продукте?", "Сколько сахара содержится в продукте?", "Пригоден ли продукт для детей?"]
for cat in choised_cats:
    print(cat, df[cat][0], sep=': ')

Категория продукта: Cухие каши и крахмалистые продукты
Наименование продукта: ЙОГУРТ "АГУША"
С какого возраста можно употреблять этот продукт?: СТАРШЕ 8 МЕСЯЦЕВ
Какие вредные вещества содержатся в продукте?: КАЛОРИЙНОСТЬ) акция проводится на территории РФ процент от суточной нормы. (Содержание молочнокислых микроорганизмов
Как называется компания изготовитель этого продукта?: ЙОГУРТ "АГУША"
Полный состав продукта: молоко нормализованное
Какая информация об этом продукте является рекламной?: КАЛОРИЙНОСТЬ
Какова энергетическая ценность продукта детского питания?: СРЕДНИЕ ЗНАЧЕНИЯ): (КАЛОРИЙНОСТЬ) акция проводится на территории РФ процент от суточной нормы
Какова пищевая ценность продукта?: ВВ12 ПИЩЕВАЯ ЦЕННОСТЬ В 100 Г ПРОДУКТА (СРЕДНИЕ ЗНАЧЕНИЯ): (КАЛОРИЙНОСТЬ
Какие подсластители и свободные сахара содержатся в продукте?: ароматизаторы натуральные ("Клубника", "Банан"), крахмал кукурузный, загуститель пектины, соки концентрированные (из красной свеклы, лимонный), пребиотик олигофруктоза